<a href="https://colab.research.google.com/github/fabrizioaymone/DeepBeliefNetwork/blob/main/dbn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow import keras

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
class rbm():
  def __init__(self, learning_rate, batch_size, epochs):
    self.toppen = np.random.normal(0, 1, (2000, 784))
    self.toplab = np.random.normal(0, 1, (2000, 10))
    self.learning_rate = learning_rate
    self.batch_size = batch_size
    self.epochs = epochs

  def train(self, X_train, y_train):
    size = X_train.shape[0]
    for epoch in range(self.epochs):
      print("Epoch number: ", epoch)
      for batch in range(size//self.batch_size):
        random_data_index = np.random.randint(X_train.shape[0], size=self.batch_size)
        X_batch = X_train[random_data_index] 
        y_batch = np.eye(10)[y_train[random_data_index]]
      
        d_toppen = 0
        d_toplab = 0
        for i in range(self.batch_size):
          #pos phase
          lab_0 = y_batch[i]
          pen_0 = X_batch[i, :]
          top_0 = np.dot(pen_0, self.toppen.T) + np.dot(lab_0, self.toplab.T)
          top_0 = self.sigmoid(top_0)
          top_0 = top_0 > np.random.random_sample(len(top_0,))
          #neg phase
          lab_1 = np.dot(top_0, self.toplab)
          pen_1 = np.dot(top_0, self.toppen)
          lab_1 = self.softmax(lab_1)
          pen_1 = self.sigmoid(pen_1)
          lab_1 = np.eye(10)[np.random.choice(range(10), 1, p=lab_1)]
          pen_1 = pen_1 > np.random.random_sample(len(pen_1,))
          
          top_1 = np.dot(pen_1, self.toppen.T) + np.dot(lab_1, self.toplab.T)
          top_1 = self.sigmoid(top_1)
          top_1 = top_1 > np.random.random_sample(len(top_1,))

          d_toppen+= (np.outer(top_0, pen_0, ) - np.outer(top_1, pen_1))/self.batch_size
          d_toplab+= (np.outer(top_0, lab_0) - np.outer(top_1, lab_1))/self.batch_size
        
        #weight update
        self.toppen += self.learning_rate * d_toppen
        self.toplab += self.learning_rate * d_toplab


  def predict(self, x_train):
    avg_lab = np.zeros(10)
    for i in range(20):
      lab_0 = np.full((10,), 0.1)
      pen_0 = x_train

      #alternating gibbs 1 reconstruction
      top_0 = np.dot(pen_0, self.toppen.T) + np.dot(lab_0, self.toplab.T)
      top_0 = self.sigmoid(top_0)
      top_0 = top_0 > np.random.random_sample(len(top_0,))

      lab_1 = np.dot(top_0, self.toplab)
      lab_1 = self.softmax(lab_1)
      avg_lab+=(lab_1/20)
    return np.argmax(avg_lab)

  def validate(self, X_val, y_val):
    size = X_val.shape[0]

    correct = 0
    for i in range(size):
      predicted = self.predict(X_val[i, :])
      print("y: ",y_val[i]," predicted: ", predicted)
      if(y_val[i] == predicted):
        correct+=1
    
    print(correct," correct on ",size," means accuracy of ",(correct/size*100),"%")


  def sigmoid(self, x):
    return 1/ (1 + np.exp(-x))
  
  def softmax(self, x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
X_train_full = X_train_full.reshape(X_train_full.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)

X_valid, X_train = X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test/255.0

In [ ]:
# try
model = rbm(0.1, 10, 30)
num = 10
X_try = X_train[1000:num+1000]
y_try = y_train[1000:num+1000]
model.train(X_try, y_try)
model.validate(X_try, y_try)

Epoch number:  0
Epoch number:  1
Epoch number:  2
Epoch number:  3
Epoch number:  4
Epoch number:  5
Epoch number:  6
Epoch number:  7
Epoch number:  8
Epoch number:  9
Epoch number:  10
Epoch number:  11
Epoch number:  12
Epoch number:  13
Epoch number:  14
Epoch number:  15
Epoch number:  16
Epoch number:  17
Epoch number:  18
Epoch number:  19
Epoch number:  20
Epoch number:  21
Epoch number:  22
Epoch number:  23
Epoch number:  24
Epoch number:  25
Epoch number:  26
Epoch number:  27
Epoch number:  28
Epoch number:  29
y:  8  predicted:  8
y:  6  predicted:  5
y:  4  predicted:  5
y:  4  predicted:  4
y:  6  predicted:  3
y:  5  predicted:  5
y:  5  predicted:  5
y:  8  predicted:  8
y:  7  predicted:  7
y:  3  predicted:  5
6  correct on  10  means accuracy of  60.0 %
